# Sales Rocket 🚀🚀🚀

Sales calls hold a wealth of untapped insights. Every transcript contains valuable signals about customer needs, objections, and pain points—but in most organizations, these records are left unused or are too cumbersome to analyze at scale. Traditional analytics tools struggle with this type of unstructured data, leaving sales teams without actionable guidance on how to improve their conversations or address recurring concerns.

This project tackles the problem by leveraging BigQuery’s advanced AI capabilities to transform raw sales call transcripts into structured insights. Using Generative AI (**ML.GENERATE_TEXT**) within BigQuery, it extracts recurring and new objections that directly impact purchasing decisions. Instead of letting transcripts collect dust, it creates a dynamic feedback loop that empowers sales teams to refine their messaging and close more deals.



#### Architecture Diagram

```text
      +----------------------------+        +------------------------------+
      |        Call transcripts    |        |          Google Cloud        |
      |        (text, CSV, JSON)   |        |    BigQuery & Vertex AI      |
      +-------------+--------------+        +---------------+--------------+
                    |  Load/ingest                          |
                    v                                       |
      +---------------------------+                         |
      | BigQuery: call_transcripts|                         |
      |           (table)         |                         |
      +-------------+-------------+                         |
                    |  Extract objections &                 |
                    |  assign categories using              v
                    |  ML.GENERATE_TEXT    +--------------------------------+   
                    |<---------------------| Vertex AI Connection           |
                    |                      | Gemini 2.5 Pro (generation)    |    
                    |                      +--------------------------------+              
                    v                                       
      +---------------------------+
      | BigQuery: objections      |
      |           (table)         |
      +-------------+-------------+
                    |
                    | Query / Dashboard / Export (e.g., BigQuery UI, BI)
                    v
      +---------------------------+
      |  Sales team / RevOps      |
      |  access insights          |
      +---------------------------+
```

## 1. Prerequisites

- **GCP Project** with BigQuery and Vertex AI services enabled
- **Model**: BigQuery [remote model](https://cloud.google.com/bigquery/docs/generate-text) powered by Vertex AI (*Gemini 2.5 Pro*)
- **Notebook Environment**: Google Colab or Kaggle (*with GCP integration*)  
- **PyPi Dependencies**:  
  - `datasets` — for loading the Hugging Face *saas-sales-conversations* dataset  
  - `google-cloud-bigquery` — for interacting with BigQuery  
  - `pandas` — for data manipulation  
  - `matplotlib` - for visiualization
- **Data**: Hugging Face [saas-sales-conversations](https://huggingface.co/datasets/DeepMostInnovations/saas-sales-conversations) datasets.


### 1.1 Installing Dependendencies

In [ ]:
!pip install pandas datasets google-cloud-bigquery matplotlib

### 1.2 GCP Project & BigQuery Settings


In [ ]:
# GCP Project ID
project_id = '<your-gcp-project-id>'

# BigQuery DataSet/Schema
dataset_id = '<exsing-bq-schema-in-above-project>'

# Call Transcripts Table Name
call_transcripts_table = 'call_transcripts'

# Objections Table
objections_table = 'objections'

# BigQuery Remote Model (format: project_id.dataset_id.model_name)
remote_model = '<your-remote-model>'

1.3 GCP Authentication for Google Colab

In [ ]:
# Obtain GCP auth token if the notebook is hosted by Colab
try:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated using Google Colab')
except:
  print('Not authenticated using Google Colab')

## 2. Data

The Hugging Face [saas-sales-conversations](https://huggingface.co/datasets/DeepMostInnovations/saas-sales-conversations) dataset is a synthetic collection of sales conversations for SaaS companies, designed for training sales conversion prediction models.

For this project, we will use a subset of sales calls related to the *DataFlow Pro* product, filtered to include only conversations with more than 15 words.  

###

### 2.1 Download saas-sales-conversations Dataset

In [ ]:
from datasets import load_dataset

# Download saas-sales-conversations dataset
dataset = load_dataset("DeepMostInnovations/saas-sales-conversations", split="train")

# Convert dataset to pandas dataframe
all_conversations_df = dataset.to_pandas()

# Data preview
all_conversations_df.head()

### 2.2 Extract Data Subset

In [ ]:
# Extract conversations for DataFlow Pro product
dataflow_pro_conversations_df = all_conversations_df[all_conversations_df['product_name'] == 'DataFlow Pro']

# Filter conversations that have over 15 words
dataflow_pro_conversations_df = dataflow_pro_conversations_df[dataflow_pro_conversations_df['conversation_length'] > 15]

# Extract "conversation_id", "scenario", "conversation" and "full_text" columns
dataflow_pro_conversations_df = dataflow_pro_conversations_df[['conversation_id', 'scenario', 'conversation', 'full_text']]

# Data preview
dataflow_pro_conversations_df.head()

### 2.3 Uplaod the Conversations to BigQuery

In [ ]:
from google.cloud import bigquery

# Create or replace call BigQuery table
dataflow_pro_conversations_df.to_gbq(
    project_id=project_id,
    destination_table=f'{dataset_id}.{call_transcripts_table}',
    if_exists='replace')

## 3. Extract Objections Using BigQuery ML

In this step, we will use BigQuery's `ML.GENERATE_TEXT` functionality to automatically identify and extract sales objections from call transcripts. By leveraging generative AI within BigQuery, we can process unstructured text data and generate structured insights that highlight recurring concerns raised during sales conversations.  

### 3.1 Create objections Table

In [ ]:
# Query to create objections table
query = f'''
DECLARE prompt_template STRING;

SET prompt_template = """You are an expert conversational analyst specializing in business to business SaaS tool sales. Your task is to analyze the provided call transcript between a Sales Executive ("sales_rep") and a prospective customer ("Customer").

Your primary objective is to analyze the call transcript that is included below. Using the information in the call transcript only your goal is to return following information:

Specific Objections: This is information regarding any objections expressed by the customer during the call. Objections may be about DataFlow Pro product and its features, or even about a potential sales contract. Provide the following for each:
  - name: a short descriptor (2 words or less) summarizing the objection.
  - concern_summary: a detailed explanation of the objection in one sentence or less.
  - category: 1 for all pricing related objections, 2 for all secuirty related objections, 3 for all integration or implementation related objections, 4 for all ROI related objections, 5 for all scalability related objections, and 6 for all other objections.


Provide your selections as a JSON output without additional commentary. Return the response strictly in the JSON format as follows with nothing else and do not label it as JSON. If there is no applicable information for any category, still include that category with an empty list. Make sure all strings are contained in quotes:
'{{
  "objections": [
    {{"name": STRING,
     "summary": STRING,
     "category": INTEGER
    }}
  ]
}}'

**Normal Output Example:**
'{{
  "objections": [
    {{"name": "Pricing"
     "summary": "The customer's voiced the objection X number of times.",
     "category": 1
    }},
    {{"name": "Integration"
     "summary": "The customer uses XYZ products and concerned about the integration process.",
     "category": 3
    }}
  ]
}}'

**Empty Output Example:**
'{{
  "objections": [
  ]
}}'

Call Transcript:

""";


-- Create objections table
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.{objections_table}`
AS

WITH transcripts AS (
  SELECT
    conversation_id,
    prompt_template || conversation as prompt
  FROM `{project_id}.{dataset_id}.{call_transcripts_table}`
),

objections_nested AS (
  SELECT
    conversation_id,
    TRIM(REGEXP_REPLACE(ml_generate_text_llm_result, "(```|json)", "")) AS generated_text
  FROM ML.GENERATE_TEXT(MODEL `{ remote_model }`,
                        TABLE transcripts,
                        STRUCT(0.2 AS temperature,
                              8000 AS max_output_tokens,
                              TRUE AS flatten_json_output))
),

objections AS (
  SELECT
    conversation_id,
    JSON_VALUE(objection, '$.name') AS objection_name,
    JSON_VALUE(objection, '$.summary') AS objection_summary,
    CAST(JSON_VALUE(objection, '$.category') AS INT64) AS objection_category
  FROM objections_nested
    INNER JOIN UNNEST(JSON_EXTRACT_ARRAY(generated_text, '$.objections')) AS objection
)

SELECT *
FROM objections;
'''

# print(query)

In [ ]:
from google.cloud import bigquery
# Create objections table
client = bigquery.Client(project=project_id)
query_job = client.query(query)
query_job.result()
print('Objections table is created successfully')

## 4. Download objections Table as a DataFame

In [ ]:
# Download objections Table as a DataFame
objections_df = client.query(f'SELECT * FROM `{project_id}.{dataset_id}.{objections_table}`').to_dataframe()

# Assign category names
category_mapping = {
    1: 'Pricing',
    2: 'Security',
    3: 'Integration',
    4: 'ROI',
    5: 'Scalibility',
    6: 'Other'
}
objections_df['objection_category'] = objections_df['objection_category'].map(category_mapping)

# Data preview
objections_df.head()

## 5. Visualize and Analyze

### 5.1 Visualize the Results

In [ ]:
import matplotlib.pyplot as plt

# Draw objections PIE chart using category field
fig, ax = plt.subplots(figsize=(10, 14))
ax.pie(objections_df['objection_category'].value_counts(), labels=objections_df['objection_category'].value_counts().index, autopct='%1.1f%%')
ax.set_title('Objections by Categories')
ax.legend(loc='upper right')
plt.show()

### 5.2 Dive Deeper into Objections in the "Other" Category

In addition to the main categories of objections, we can analyze the transcripts labeled as **"Other"** to uncover less common but potentially critical concerns. By understanding these overlooked objections, sales teams can prepare targeted response scripts and ensure they are equipped to handle a wider range of scenarios during future conversations.  

In [ ]:
# Filter objections in "Other" category
objections_df[objections_df['objection_category'] == 'Other'].sort_values(by='objection_name')